<a href="https://colab.research.google.com/github/Narean-AI/Retail-store-purchase-history/blob/main/fake_currency_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
from PIL import Image

# Create dummy directories
os.makedirs('data/real', exist_ok=True)
os.makedirs('data/fake', exist_ok=True)

# Generate dummy images
def create_dummy_image(path, size=(128, 128)):
    img_array = np.random.randint(0, 256, size + (3,), dtype=np.uint8)
    img = Image.fromarray(img_array)
    img.save(path)

for i in range(50):
    create_dummy_image(f'data/real/real_{i}.png')
    create_dummy_image(f'data/fake/fake_{i}.png')

print("Dummy dataset created.")

Dummy dataset created.


In [ ]:
import cv2
from sklearn.model_selection import train_test_split

image_size = (128, 128)
images = []
labels = []

# Load real images
for filename in os.listdir('data/real'):
    img = cv2.imread(os.path.join('data/real', filename))
    img = cv2.resize(img, image_size)
    images.append(img)
    labels.append(0) # 0 for real

# Load fake images
for filename in os.listdir('data/fake'):
    img = cv2.imread(os.path.join('data/fake', filename))
    img = cv2.resize(img, image_size)
    images.append(img)
    labels.append(1) # 1 for fake

images = np.array(images, dtype=np.float32)
labels = np.array(labels)

# Normalize pixel values
images /= 255.0

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

Training set size: 80
Testing set size: 20


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Apply data augmentation to the training set
datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Fit the data augmentation generator on the training data
datagen.fit(X_train)

print("Data augmentation applied to the training set.")

Data augmentation applied to the training set.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Create a Sequential model
model = Sequential()

# Add a Conv2D layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))

# Add a MaxPooling2D layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add another Conv2D layer
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another MaxPooling2D layer
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a Flatten layer
model.add(Flatten())

# Add a Dense layer
model.add(Dense(128, activation='relu'))

# Add a Dropout layer
model.add(Dropout(0.5))

# Add the final Dense output layer
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,392,449 (28.20 MB)

 Trainable params: 7,392,449 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    epochs=20,
                    validation_data=(X_test, y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 914ms/step - accuracy: 0.5464 - loss: 1.6998 - val_accuracy: 0.6000 - val_loss: 2.4977
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 679ms/step - accuracy: 0.4745 - loss: 2.1713 - val_accuracy: 0.4000 - val_loss: 1.4919
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 2s/step - accuracy: 0.5437 - loss: 0.9049 - val_accuracy: 0.4000 - val_loss: 0.7473
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.5771 - loss: 0.6705 - val_accuracy: 0.4000 - val_loss: 0.6937
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 883ms/step - accuracy: 0.4865 - loss: 0.6930 - val_accuracy: 0.6000 - val_loss: 0.6931
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 651ms/step - accuracy: 0.5406 - loss: 0.6930 - val_accuracy: 0.5500 - val_loss: 0.6930
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 856ms/step - accuracy: 0.6052 - loss: 0.6927 - val_accuracy: 0.3500 - val_loss: 0.6941
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 843ms/step - accuracy: 0.4526 - loss: 0.6937 - val_accuracy: 0.4000 - val_loss: 0.699

In [ ]:
from sklearn.metrics import precision_score, recall_score

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

# Print the obtained loss and accuracy
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Predict the probabilities of the test set
y_pred_prob = model.predict(X_test)

# Convert the predicted probabilities to binary class labels
y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate precision and recall
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

# Print the calculated precision and recall
print(f"Test Precision: {precision:.4f}")
print(f"Test Recall: {recall:.4f}")

Test Loss: 0.6952
Test Accuracy: 0.4000
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 888ms/step
Test Precision: 0.4000
Test Recall: 1.0000


In [ ]:
import cv2
import numpy as np
from PIL import Image

# Create a dummy image for prediction
new_image_path = 'data/new_currency.png'
create_dummy_image(new_image_path)
print(f"Dummy new image created at: {new_image_path}")

# Load the new image
new_image = cv2.imread(new_image_path)

# Preprocess the new image
new_image = cv2.resize(new_image, image_size)
new_image = new_image.astype(np.float32) / 255.0
new_image = np.expand_dims(new_image, axis=0) # Add batch dimension

# Use the trained model to predict
prediction = model.predict(new_image)

# Classify based on the threshold
if prediction[0] > 0.5:
    predicted_class = "Fake"
else:
    predicted_class = "Real"

# Print the predicted class
print(f"The new currency image is predicted as: {predicted_class}")

Dummy new image created at: data/new_currency.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step
The new currency image is predicted as: Fake
